<a href="https://colab.research.google.com/github/lorenafc/Capita_Selecta_RHI50403/blob/main/geocoding_GoogleAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install googlemaps -q


  Preparing metadata (setup.py) ... done


In [ ]:
import googlemaps
import pandas as pd
import time
import logging
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# upload the file
from google.colab import files
uploaded=files.upload()

Saving merged_not_geocoded.xlsx to merged_not_geocoded.xlsx


In [ ]:
file_name = 'file_not_geocoded.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')



In [ ]:
smalldataset_ten_thousand_cleaned = authors_small_cities.copy()

In [1]:
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Google Maps client with your API key
API_KEY = 'YOUR_GOOGLE_API_KEY'
gmaps = googlemaps.Client(key=API_KEY)

# Function to geocode city names with error handling and rate limiting
def geocode_city(city_name):
    try:
        # Rate limiting
        time.sleep(1)
        # Attempt to geocode the city name with a timeout
        geocode_result = gmaps.geocode(city_name)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return (city_name, location['lat'], location['lng'])
        else:
            logging.warning(f"Geocoding failed for: {city_name}")
            return (city_name, None, None)
    except Exception as e:
        logging.error(f"Geocoding error for {city_name}: {e}")
        return (city_name, None, None)

# Function to geocode a column of city names and map the results back to the DataFrame
def geocode_column(df, column_name, lat_col_name, lon_col_name):
    # Filter out rows where the column is NaN
    df_filtered = df.dropna(subset=[column_name])

    # Extract unique city names from the DataFrame column
    unique_cities = list(set(df_filtered[column_name]))

    # Geocode unique city names with progress tracking
    geocoded_results = [geocode_city(city) for city in tqdm(unique_cities, desc=f'Geocoding {column_name}')]

    # Create a dictionary to map city names to their geocoded coordinates
    geocode_dict = {result[0]: (result[1], result[2]) for result in geocoded_results}

    # Map geocoded coordinates back to the original DataFrame
    df[f'{column_name}_coordinates'] = df[column_name].map(geocode_dict)

    # Ensure coordinates are always tuples with two elements
    df[f'{column_name}_coordinates'] = df[f'{column_name}_coordinates'].apply(lambda x: (x if isinstance(x, tuple) else (None, None)))

    # Extract latitude and longitude from coordinates, filling NaN where necessary
    df[[lat_col_name, lon_col_name]] = df[f'{column_name}_coordinates'].apply(pd.Series)

    # Drop the coordinates column
    df.drop(columns=[f'{column_name}_coordinates'], inplace=True)

    return df

# Geocode activecity, borncity, and deathcity columns with distinct names for latitude and longitude
columns_to_geocode = [
    ('borncity', 'latitude_born', 'longitude_born'),
    ('deathcity', 'latitude_death', 'longitude_death'),
    ('activecity', 'latitude_active', 'longitude_active')
]

# Assuming smalldataset_ten_thousand_cleaned is your DataFrame
for column, lat_col, lon_col in columns_to_geocode:
    smalldataset_ten_thousand_cleaned = geocode_column(smalldataset_ten_thousand_cleaned, column, lat_col, lon_col)

# Display the df with geocoded coordinates
print(smalldataset_ten_thousand_cleaned.head(15))


# Specify the path in Google Drive
file_path = '/content/drive/My Drive/geocoded_GoogleAPI.xlsx'

# Save also the df to an Excel file in Google Drive
smalldataset_ten_thousand_cleaned.to_excel(file_path)

# export the result as excel file
file_name = 'authors_cities_geocoded_GoogleAPI.xlsx'
smalldataset_ten_thousand_cleaned.to_excel(file_name)


NameError: name 'logging' is not defined